In [ ]:
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset

class LandmarksDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = Path(root_dir)
        self.samples = []
        self.class_names = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {c: i for i, c in enumerate(self.class_names)}

        for cls in self.class_names:
            for f in (self.root_dir / cls).glob("*.npy"):
                self.samples.append((f, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        x = np.load(path).astype("float32")
        x = torch.from_numpy(x)
        y = torch.tensor(label, dtype=torch.long)
        return x, y
# Example usage:
train_ds = LandmarksDataset("data/landmarks/train")
val_ds   = LandmarksDataset("data/landmarks/val")

In [7]:
import torch.nn as nn

class LandmarksMLP(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(63, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)


In [10]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss


BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

train_ds = LandmarksDataset("data/landmarks/train")
val_ds   = LandmarksDataset("data/landmarks/val")

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)

NUM_CLASSES = len(train_ds.class_names)

model = LandmarksMLP(NUM_CLASSES).to(DEVICE)
optimizer = Adam(model.parameters(), lr=LR)
criterion = CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # VALIDATION
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | loss={total_loss:.3f} | val_acc={acc:.3f}")

# SAVE
torch.save(model.state_dict(), "checkpoints/landmarks_mlp.pt")


Epoch 1/30 | loss=60.818 | val_acc=0.610
Epoch 2/30 | loss=40.567 | val_acc=0.755
Epoch 3/30 | loss=29.007 | val_acc=0.824
Epoch 4/30 | loss=21.933 | val_acc=0.848
Epoch 5/30 | loss=17.206 | val_acc=0.879
Epoch 6/30 | loss=14.191 | val_acc=0.885
Epoch 7/30 | loss=12.220 | val_acc=0.904
Epoch 8/30 | loss=10.373 | val_acc=0.904
Epoch 9/30 | loss=9.360 | val_acc=0.885
Epoch 10/30 | loss=8.098 | val_acc=0.923
Epoch 11/30 | loss=7.656 | val_acc=0.907
Epoch 12/30 | loss=6.837 | val_acc=0.920
Epoch 13/30 | loss=6.314 | val_acc=0.923
Epoch 14/30 | loss=6.134 | val_acc=0.910
Epoch 15/30 | loss=5.729 | val_acc=0.935
Epoch 16/30 | loss=5.429 | val_acc=0.947
Epoch 17/30 | loss=4.985 | val_acc=0.932
Epoch 18/30 | loss=5.090 | val_acc=0.938
Epoch 19/30 | loss=5.015 | val_acc=0.935
Epoch 20/30 | loss=4.399 | val_acc=0.929
Epoch 21/30 | loss=4.558 | val_acc=0.916
Epoch 22/30 | loss=4.318 | val_acc=0.938
Epoch 23/30 | loss=4.133 | val_acc=0.941
Epoch 24/30 | loss=3.703 | val_acc=0.944
Epoch 25/30 | los